In [1]:
import numpy as np

$$
{\rm d}\boldsymbol{x}(t)=[-\boldsymbol{x}(t)+\boldsymbol{W}\cdot h(\boldsymbol{x}(t))]\cdot{\rm d}t+\sigma\cdot{\rm d}\boldsymbol{\xi}(t)
$$

$$
\boldsymbol{x}(t+\Delta t) \approx \boldsymbol{x}(t) + [-\boldsymbol{x}(t)+\boldsymbol{W}\cdot h(\boldsymbol{x}(t))]\cdot\Delta t + \sigma\sqrt{\Delta t}\cdot\boldsymbol{Z}(t),\,
\boldsymbol{Z}(t)\sim\mathcal{N}(\boldsymbol{0},\,\boldsymbol{I}_n)
$$

In [2]:
dt          = 0.01
TR          = 1
down_sample = int(TR/dt)
sigma       = 10

In [3]:
def h(x):
    return np.tanh(x)

def NextTR(W, X_init, noise):
    X_res = X_init.copy()
    for i in range(down_sample): X_res = X_res + (-X_res + np.dot(W.T, h(X_res))) * dt + noise[i]
    return X_res

def RNN(W, X_init, length, pert_strength = 1.0, pert_step = 200):
    np.random.seed(0)
    n = len(X_init); X = [X_init]
    real_EC = np.zeros((n, n)); pert_count = 0
    for i in range(1, length):
        noise = (sigma * dt) * np.random.normal(0, 1, (down_sample, n))
        X.append(NextTR(W, X[-1], noise))
        if i % pert_step == 0:
            pert_count += 1
            for j in range(n):
                perted_init = X[-2].copy(); perted_init[j] += pert_strength
                real_EC[j] += NextTR(W, perted_init, noise) - X[-1]
    real_EC /= pert_count; np.fill_diagonal(real_EC, 0)
    return np.array(X), real_EC

### **RNN simulation**

##### generate RNN synthetic neural data

In [4]:
node_n      = 20
data_length = 8000

In [5]:
for random_seed in range(50):

    np.random.seed(random_seed)
    SC = np.random.normal(0, 1/np.sqrt(node_n), (node_n, node_n)); np.fill_diagonal(SC, 0)
    init = np.random.normal(0, 1, (node_n))
    signal, real_EC = RNN(SC, init, data_length)

    np.savetxt('./sim_data/SC_{:03d}nodes_'.format(node_n) + '{:04d}steps_'.format(data_length) + '{:02d}seed.txt'.format(random_seed), SC)
    np.savetxt('./sim_data/dynamics_{:03d}nodes_'.format(node_n) + '{:04d}steps_'.format(data_length) + '{:02d}seed.txt'.format(random_seed), signal)
    np.savetxt('./sim_data/real_EC_{:03d}nodes_'.format(node_n) + '{:04d}steps_'.format(data_length) + '{:02d}seed.txt'.format(random_seed), real_EC)